# mp3_to_text.ipynb file making


istall openai-whisper


In [1]:
!pip install -U openai-whisper


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 12.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.3/188.3 MB 6.1 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=e19dda6d9433d6f679c2d61798acbd6dd8e8e27c1bf8a1a21d848cfd1571b50e
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
Successfully built openai-whisper


### Mount your drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Making the .mp3 file to json file

 ## all store in chunks

In [5]:
import whisper
import json
import os
os.makedirs("/content/drive/MyDrive/RAG_BAS_PROJECT/jsons", exist_ok=True)

# model = whisper.load_model("large-v2")

audios = os.listdir("/content/drive/MyDrive/RAG_BAS_PROJECT/AUDIOS")


for audio in audios:
  number = audio.split("_")[0]
  title =  audio.split("_")[1][:-4]
  result=model.transcribe(audio = f"/content/drive/MyDrive/RAG_BAS_PROJECT/AUDIOS/{audio}",
                        language = 'en',
                        task="transcribe",
                        word_timestamps=False)
  chunks = []
  for segment in result["segments"]:
      chunks.append({
      "Number":number,
      "Title":title ,
      "Start":segment["start"],
      "End": segment["end"] ,
      "Text":segment["text"].strip()
      })

  chunks_with_metadata = {"chunks": chunks, "text": result["text"]}

  with open(f"/content/drive/MyDrive/RAG_BAS_PROJECT/jsons/{number}_{title}.json","w")as f:
     json.dump(chunks_with_metadata, f, indent=2)







# Install Sentence Transformers for Embeddings

In [3]:
!pip install sentence-transformers


# Create Embeddings and store DataFrame

In [5]:
import os
import json
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
device = "cuda" if torch.cuda.is_available() else "cpu"

model = SentenceTransformer("BAAI/bge-m3",
                               device= device)
def create_embeddings(text_list,batch_size = 8):


    embedding = model.encode(
    text_list,
    batch_size=batch_size,
    normalize_embeddings=True,
    show_progress_bar=False
      )

    return embedding;

# a = create_embeddings("cdat sat on the mat.")
# print(a)
jsons = os.listdir("/content/drive/MyDrive/RAG_BAS_PROJECT/jsons")
my_dicts = []
chunk_id = 0
for json_file in jsons:
  with open(f"/content/drive/MyDrive/RAG_BAS_PROJECT/jsons/{json_file}")as f:
    content = json.load(f)
    print(f"creating embeddings for {json_file}")
    embeddings = create_embeddings([c["Text"]for c in content["chunks"]])

    for i ,chunk in enumerate(content["chunks"]):
      chunk["chunk_id"] = chunk_id
      chunk["embedding"] = embeddings[i].tolist()
      chunk_id += 1
      my_dicts.append(chunk)
    torch.cuda.empty_cache()

df = pd.DataFrame.from_records(my_dicts)
print(df)





/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

creating embeddings for 01_Introduction to Signals and Systems.json
creating embeddings for 02_Continuous and Discrete Time Signals.json
creating embeddings for 03_Addition of Continuous-Time Signals.json
creating embeddings for 04_Multiplication of Continuous-Time Signals.json
creating embeddings for 05_Time Scaling of Continuous-Time Signals.json
creating embeddings for 06_Amplitude Scaling of Continuous-Time Signals.json
creating embeddings for 07_Time Shifting of Continuous-Time Signals.json
creating embeddings for 08_Amplitude Shifting of Continuous-Time Signals.json
creating embeddings for 09_Reversal of Continuous-Time Signals.json
creating embeddings for 10_Multiple Transformations of Continuous-Time Signals.json
creating embeddings for 11_Multiple Transformations of CTS (Important Point & Shortcut).json
creating embeddings for 12_Multiple Transformations of CTS (Solved Problem 1).json
creating embeddings for 13_Multiple Transformations of CTS (Solved Problem 2).json
creating e

 # convert embedding list to JSON string to store in .csv file
 # save in .csv file

In [6]:
# convert embedding list → JSON string
df["embedding"] = df["embedding"].apply(json.dumps)

# save to CSV
df.to_csv(
    "/content/drive/MyDrive/RAG_BAS_PROJECT/embeddings.csv",
    index=False
)

print("CSV saved successfully!")


CSV saved successfully!


# Read the csv

In [7]:
import json
import pandas as pd
import numpy as np
df = pd.read_csv("/content/drive/MyDrive/RAG_BAS_PROJECT/embeddings.csv")





# Make a copy of the .csv file (for not effecting in original csv )

In [8]:
df1=df.copy()
print(df1["embedding"].values)

['[-0.028110971674323082, 0.004526797216385603, -0.049466557800769806, 0.021494174376130104, -0.009733356535434723, -0.016012100502848625, 0.008170937187969685, 0.003943813499063253, -0.004198470152914524, 0.017138976603746414, 0.020365014672279358, -0.016674209386110306, -0.01945667527616024, -0.0013734120875597, -0.00855968240648508, 0.0040671504102647305, 0.04194840416312218, -0.01687757484614849, -0.0038171978667378426, 0.004774657543748617, 0.02589796483516693, -0.006202127318829298, 0.001516574528068304, 0.05329354107379913, 0.009225087240338326, 0.02772039733827114, -0.029477674514055252, -0.01893872208893299, -0.0030966324266046286, 0.004231145605444908, 0.025549178943037987, 0.0022264765575528145, 0.01512407511472702, -0.008850307203829288, -0.019661694765090942, -0.04807204380631447, 0.03902827575802803, -0.0316031351685524, -0.04488084837794304, 0.005182122811675072, -0.01487597357481718, -0.027779221534729004, 0.002929763635620475, -0.0922260731458664, 0.023454096168279648,

# Convert embedding json string to list by apply json.loads(string → numeric list)



In [9]:
df1["embedding"] = df1["embedding"].apply(json.loads)
#print(np.vstack(df1["embedding"][0]))
print(np.vstack(df1["embedding"]).shape)


(1209, 1024)


# Taking user input and create the embedding of the input and by use of sklearn cosine_similarity check the similarity between the input and the embedding of all over data set

In [10]:
import os
import json
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

device = "cuda" if torch.cuda.is_available() else "cpu"

# model = SentenceTransformer("BAAI/bge-m3",
#                                device= device)
def create_embeddings(text_list,batch_size = 8):


    embedding = model.encode(
    text_list,
    batch_size=batch_size,
    normalize_embeddings=True,
    show_progress_bar=False
      )

    return embedding;

incoming_query = input("Ask Any Question:")
question_embedding = create_embeddings([incoming_query])
similarities = cosine_similarity(np.vstack(df1["embedding"]),question_embedding).flatten()
# print(similarities)

Ask Any Question:what is signal?


# Take top 3 result

In [11]:
top_result = 3
max_index = similarities.argsort()[::-1][0:top_result]
# print(max_index)
new_def = df1.iloc[max_index]
# print(new_def)
# print(new_def[["Title","Number","Start","End","Text"]])

# Gemini API key configured!

In [13]:
!pip install -q -U google-generativeai

import google.generativeai as genai
from google.colab import userdata

# Load the API key from Colab Secrets
api_key = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=api_key)

print("Gemini API key configured!")


Gemini API key configured!


### Define a function to interact with the Gemini API for Retrieval Augmented Generation (RAG)

In [14]:
def inference_gemini(context_df, user_query):
    context_text = ""

    for _, row in context_df.iterrows():
        context_text += f"""
Lecture Title: {row['Title']}
Lecture Number: {row['Number']}
Start Time: {row['Start']} seconds
End Time: {row['End']} seconds
Lecture Text:
{row['Text']}
--------------------------------
"""

    prompt = f"""
You are an AI teaching assistant for a Signals and Systems course.

IMPORTANT RULES:
- Use ONLY the lecture content below.
- You are ALLOWED to combine information from multiple lecture chunks.
- The lecture content may be introductory or partial — still answer if the concept is being introduced or explained.
- Only say "This question is not covered in the provided lectures."
  if the topic is COMPLETELY unrelated.

You MUST:
- Explain the concept in very simple student-friendly language
- Mention ALL relevant:
  - Lecture Title
  - Lecture Number
  - Start Time
  - End Time

Lecture Content:
{context_text}

User Question:
{user_query}

Answer Format (STRICT):

Answer:
<simple explanation derived from lecture content>

Where it is taught:
- Lecture Title:
- Lecture Number:
- Start Time:
- End Time:
"""

    model = genai.GenerativeModel("gemini-2.5-flash")
    response = model.generate_content(prompt)

    return response.text


# Show the answer

In [15]:
# incoming_query = input("Ask any question: ")

response_text = inference_gemini(new_def, incoming_query)

print("AI Answer:\n")
print(response_text)


AI Answer:

Answer:
Based on the provided lecture content, a 'signal' is introduced as a fundamental concept in Signals and Systems. The lectures pose the question, "what is a signal?", and highlight that its definition is important and covered within the course.

Where it is taught:
- Lecture Title: Introduction to Signals and Systems
- Lecture Number: 1
- Start Time: 101.5 seconds
- End Time: 103.2 seconds
- Lecture Title: Introduction to Signals and Systems
- Lecture Number: 1
- Start Time: 279.58 seconds
- End Time: 284.34 seconds



# Save the response



In [16]:
with open("/content/drive/MyDrive/RAG_BAS_PROJECT/response.txt", "w") as f:
    f.write(response_text)
